# 데이터 준비하기: 다듬기, 변형, 병합

## 데이터 합치기

- pandas.merge는 하나 이상의 키를 기준으로 DataFrame의 로우를 합친다.
    - SQL이나 다른 관계형 데이터 페이스의 join과 유사하다.
- pandas.concat은 하나의 축을 따라 객체를 이어붙인다.
- combine_first 인스턴스 메서드는 구 객체를 포개서 한 객체에서 누락된 데이터를 다른 객체에 있는 값으로 채울 수 있도록 한다.

## 데이터베이스 스타일로 DataFrame 합치기

In [1]:
from pandas import Series, DataFrame
import pandas as pd
import numpy as np

In [2]:
df1 = DataFrame({'key': ['b', 'b', 'a', 'c', 'a', 'a', 'b'],
                 'data1': range(7)})

In [3]:
df2 = DataFrame({'key': ['a', 'b', 'd'], 'data2': range(3)})

In [4]:
df1

,data1,key
0,0,b
1,1,b
2,2,a
3,3,c
4,4,a
5,5,a
6,6,b


In [5]:
df2

,data2,key
0,0,a
1,1,b
2,2,d


- 일 대 다의 예제다.
- df1 데이터는 key 칼럼에 여러 개의 a, b를 가지고 있고 df2의 key 칼럼은 유일한 로우를 가지고 있다.

In [6]:
pd.merge(df1, df2)

,data1,key,data2
0,0,b,1
1,1,b,1
2,6,b,1
3,2,a,0
4,4,a,0
5,5,a,0


- 어떤 칼럼을 병합할 것인지 명시하지 않았다.
- merge 함수는 겹치는 칼럼의 이름을 키로 사용한다.
- 키를 명시적으로 지정하는 습관을 들이는 것이 좋다.

In [7]:
pd.merge(df1, df2, on='key')

,data1,key,data2
0,0,b,1
1,1,b,1
2,6,b,1
3,2,a,0
4,4,a,0
5,5,a,0


- 두 객체에 공통되는 칼럼 이름이 하나도 없다면 따로 지정할 수 있다.

In [8]:
df3 = DataFrame({'lkey': ['b', 'b', 'a', 'c', 'a', 'a', 'b'],
                 'data1': range(7)})

In [9]:
df4 = DataFrame({'rkey': ['a', 'b', 'd'], 'data2': range(3)})

In [10]:
pd.merge(df3, df4, left_on='lkey', right_on='rkey')

,data1,lkey,data2,rkey
0,0,b,1,b
1,1,b,1,b
2,6,b,1,b
3,2,a,0,a
4,4,a,0,a
5,5,a,0,a


- merge 함수는 기본적으로 inner join을 수행하여 교집합인 결과를 반환한다.
- how 인자로 left, right, outer를 넘겨서 각각 왼쪽 우선 외부조인, 오른쪽 우선 외부조인, 완전 외부 조인을 수행할 수 있다.
- 완전 외부조인은 합집합인 결과를 반환한다.
- 외쪽 우선 외부조인과 오른쪽 우선 외부조인은 각각 왼쪽 혹은 오른쪽의 모든 로우를 포함하는 결과를 반환한다.

In [11]:
pd.merge(df1, df2, how='outer')

,data1,key,data2
0,0.0,b,1.0
1,1.0,b,1.0
2,6.0,b,1.0
3,2.0,a,0.0
4,4.0,a,0.0
5,5.0,a,0.0
6,3.0,c,NaN
7,NaN,d,2.0


In [12]:
pd.merge(df1, df2, how='left')

,data1,key,data2
0,0,b,1.0
1,1,b,1.0
2,2,a,0.0
3,3,c,NaN
4,4,a,0.0
5,5,a,0.0
6,6,b,1.0


In [13]:
pd.merge(df1, df2, how='right')

,data1,key,data2
0,0.0,b,1
1,1.0,b,1
2,6.0,b,1
3,2.0,a,0
4,4.0,a,0
5,5.0,a,0
6,NaN,d,2


In [14]:
pd.merge(df1, df2, how='inner')

,data1,key,data2
0,0,b,1
1,1,b,1
2,6,b,1
3,2,a,0
4,4,a,0
5,5,a,0


- 다 대 다 병합은 잘 정의되어 있긴 하지만 직관적이지는 않다.

In [15]:
df1 = DataFrame({'key': ['b', 'b', 'a', 'c', 'a', 'b'],
                 'data1': range(6)})

In [16]:
df2 = DataFrame({'key': ['a', 'b', 'a', 'b', 'd'],
                 'data2': range(5)})

In [17]:
df1

,data1,key
0,0,b
1,1,b
2,2,a
3,3,c
4,4,a
5,5,b


In [18]:
df2

,data2,key
0,0,a
1,1,b
2,2,a
3,3,b
4,4,d


In [19]:
pd.merge(df1, df2, on='key', how='left')

,data1,key,data2
0,0,b,1.0
1,0,b,3.0
2,1,b,1.0
3,1,b,3.0
4,2,a,0.0
5,2,a,2.0
6,3,c,NaN
7,4,a,0.0
8,4,a,2.0
9,5,b,1.0


In [20]:
pd.merge(df1, df2, on='key', how='right')

,data1,key,data2
0,0.0,b,1
1,1.0,b,1
2,5.0,b,1
3,0.0,b,3
4,1.0,b,3
5,5.0,b,3
6,2.0,a,0
7,4.0,a,0
8,2.0,a,2
9,4.0,a,2


- 다 대 다 조인은 두 로우의 데카르트 곱을 반환한다.
- df1에는 b 로우가 3개, df2에는 b 로우가 2개, 결과는 6개의 b 로우가 된다.

In [21]:
pd.merge(df1, df2, how='inner')

,data1,key,data2
0,0,b,1
1,0,b,3
2,1,b,1
3,1,b,3
4,5,b,1
5,5,b,3
6,2,a,0
7,2,a,2
8,4,a,0
9,4,a,2


In [22]:
left = DataFrame({'key1': ['foo', 'foo', 'bar'],
                  'key2': ['one', 'two', 'one'],
                  'lval': [1, 2, 3]})

In [23]:
right = DataFrame({'key1': ['foo', 'foo', 'bar', 'bar'],
                   'key2': ['one', 'one', 'one', 'two'],
                   'rval': [4, 5, 6, 7]})

In [24]:
pd.merge(left, right, on=['key1', 'key2'], how='outer')

,key1,key2,lval,rval
0,foo,one,1.0,4.0
1,foo,one,1.0,5.0
2,foo,two,2.0,NaN
3,bar,one,3.0,6.0
4,bar,two,NaN,7.0
